In [35]:
import pyvista as pv
import numpy as np
from typing import Any, Tuple
from importlib import reload
import multiprocessing
import ifcopenshell
import ifcopenshell.geom
import time
from functools import reduce
import open3d as o3d
import random
import glob
import os
import pandas as pd
from collections import defaultdict
import datetime
from csv import writer
import traceback
import concurrent.futures

In [36]:
def random_color():
    """Generate a random RGB color."""
    return [random.random(), random.random(), random.random()]

def open3d_block_by_element(ifc_file): 
    # Configuration for ifcopenshell geometric settings
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)
    settings.set(settings.APPLY_DEFAULT_MATERIALS, True)

    # Initialize the iterator for geometric representations
    iterator = ifcopenshell.geom.iterator(settings, ifc_file, multiprocessing.cpu_count())
    
    # Define entity types to include and exclude
    exclude_list = set(["IfcCovering", "IfcFurnishingElement", "IfcSpace", "IfcOpening", "IfcOpeningElement", "IfcRailing"])
    
    # Lists to store the resulting meshes and element information
    all_meshes = []
    element_information = {}
    dims = []

    # Variable to track the smallest dimension among all meshes
    temp_dimension_x = 0.8
    temp_dimension_y = 0.8
    temp_dimension_z = 0.8
    
    # Iterate over the geometric representations
    if iterator.initialize():
        index = 0
        while True:
            shape = iterator.get()
            
            # Check if the shape type is not excluded and represents an IfcBuildingElement
            if shape.type not in exclude_list and shape.product.is_a("IfcBuildingElement"):
                # Convert the shape's geometry to an Open3D mesh
                faces = np.array(shape.geometry.faces)
                verts = np.array(shape.geometry.verts).reshape(-1, 3)
                mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(verts),
                                                 triangles=o3d.utility.Vector3iVector(faces.reshape(-1, 3)))
                
                # Update the smallest dimension if necessary
                dimensions = mesh.get_max_bound() - mesh.get_min_bound()
                dims.append(dimensions)
                min_dimension_x, min_dimension_y, min_dimension_z = dimensions

                # Update temp_dimension_x, temp_dimension_y, temp_dimension_z if necessary
                if min(temp_dimension_x, min_dimension_x) > 0.1:
                    temp_dimension_x = min(temp_dimension_x, min_dimension_x)
                if min(temp_dimension_y, min_dimension_y) > 0.1:
                    temp_dimension_y = min(temp_dimension_y, min_dimension_y)

                # Store the mesh and its corresponding IFC GUID
                all_meshes.append(mesh)
                element_information[index] = shape.guid
                index += 1
                
                # Check if the shape is a slab and update the maximum x dimension if necessary
                if shape.type == "IfcSlab":
                    if min(temp_dimension_z, min_dimension_z) >= 0.1:
                        temp_dimension_z = min(temp_dimension_z, min_dimension_z)/2                               
                    
            # Move to the next geometric representation
            if not iterator.next():
                
                break

    return all_meshes, element_information, round(temp_dimension_x,2), round(temp_dimension_y,2), round(temp_dimension_z,2), dims

def create_uniform_grid(bounds, voxel_size_x, voxel_size_y, voxel_size_z):
    """Create a uniform grid within the given bounds with rectangular cells."""
    x = np.arange(bounds[0], bounds[1] + voxel_size_x, voxel_size_x)
    y = np.arange(bounds[2], bounds[3] + voxel_size_y, voxel_size_y)
    z = np.arange(bounds[4], bounds[5] + voxel_size_z, voxel_size_z)
    return pv.StructuredGrid(*np.meshgrid(x, y, z))

def open3d_to_pyvista(point_cloud_o3d):
    """
    Convert an Open3D point cloud to a PyVista point cloud.

    Parameters:
    - point_cloud_o3d: The Open3D point cloud.

    Returns:
    - A PyVista `PolyData` object.
    """
    # Extract points from Open3D point cloud
    points = np.asarray(point_cloud_o3d.points)

    # Create a PyVista PolyData object
    point_cloud_pv = pv.PolyData(points)

    return point_cloud_pv

def process_point(pcd, mesh_guid_index, grid_bounds, x_size, y_size, z_size, dims, grid_attributes):
    j = int((pcd[0] - grid_bounds[0]) / x_size)
    i = int((pcd[1] - grid_bounds[2]) / y_size)
    k = int((pcd[2] - grid_bounds[4]) / z_size)
    
    if 0 <= i < dims[0]-1 and 0 <= j < dims[1]-1 and 0 <= k < dims[2]-1 and grid_attributes[i,j,k] == -1:
        grid_attributes[i,j,k] = mesh_guid_index

def voxelize_space(bounds, pcd_list, x_size, y_size, z_size):
    """Create a 3d grid and check the intersections of the meshes with the grid."""
    grid = create_uniform_grid(bounds, x_size, y_size, z_size)
    
    # create an empty 3d array in the same dimensions of the grid
    dims = grid.dimensions
    grid_attributes = np.full((dims[0]-1, dims[1]-1, dims[2]-1), -1, dtype=int)

    grid_bounds = grid.bounds

    [process_point(point, i, grid_bounds, x_size, y_size, z_size, dims, grid_attributes) 
    for i, pcd in enumerate(pcd_list) for point in pcd.points]
 
    # assign empty array to the grid
    grid.cell_data['attributes'] = grid_attributes.flatten(order='F').astype(int)
    
    return grid

def get_sampling_points(mesh, voxel_size, points_per_unit_area=120):
    
    # Calculate the surface area of the mesh
    area = mesh.get_surface_area()

    # Calculate the number of points to sample
    N = int(area * points_per_unit_area*1/voxel_size)
        
    return N

def create_point_cloud(all_meshes, x_size):
    """Create a point cloud from a mesh."""

    pcd_list = []

    for mesh in all_meshes:
        # Get the number of points to sample based on the mesh size
        N = get_sampling_points(mesh, x_size)
        pcd = mesh.sample_points_uniformly(N)
        pcd_pv = open3d_to_pyvista(pcd)
        pcd_list.append(pcd_pv)  
        
    return pcd_list


def fetch_element_quantities(ifc_file, grid, element_information):
    # Fetch all property sets named 'ZECH_ATTRIBUTE'
    start_time = time.time()
    print(f'Calculating quantities in {ifc_file}...')
    property_sets = ifc_file.by_type("IfcPropertySet")
    zech_property_sets = [ps for ps in property_sets if ps.Name == "ZECH_ATTRIBUTE"]
    
    # Create a mapping of elements to their ZB_AUSWAHL values
    element_to_zb_auswahl = {}
    for ps in zech_property_sets:
        for prop in ps.HasProperties:
            if prop.is_a("IfcPropertySingleValue") and prop.Name == "ZB_AUSWAHL":
                # Find all elements that have this property set
                for rel in ifc_file.by_type("IfcRelDefinesByProperties"):
                    if rel.RelatingPropertyDefinition == ps:
                        for related_object in rel.RelatedObjects:
                            element_to_zb_auswahl[related_object.GlobalId] = str(prop.NominalValue.wrappedValue)
    
    element_counts = defaultdict(int)

    # Calculate counts and collect x, y, z values for cells matching the attribute
    for i, att in enumerate(grid.cell_data['attributes']):
        element_guid = element_information.get(att)
        zb_auswahl_value = element_to_zb_auswahl.get(element_guid)
        if zb_auswahl_value is not None:
            element_counts[zb_auswahl_value] += 1

    results_QTO = {}
     # Create a mapping of ZB_AUSWAHL values to their building element types
    zb_auswahl_to_element_type = {}
    for element_guid, zb_auswahl_value in element_to_zb_auswahl.items():
        element = ifc_file.by_guid(element_guid)
        if element:
            element_type = element.is_a()
            zb_auswahl_to_element_type[zb_auswahl_value] = element_type

    # Store the counts for different ZB_AUSWAHL values
    for zb_auswahl_value, count in element_counts.items():
        results_QTO[f'{zb_auswahl_value}_count'] = count

    end_time = time.time()
    quantity_calculation_time = end_time - start_time
    results_QTO['Quantity Calculation (sec)'] = round(quantity_calculation_time,2)
    results_QTO['ZB_AUSWAHL_to_Element_Type'] = zb_auswahl_to_element_type

    return results_QTO

def process_ifc_file(file_name, x_size=0.1, y_size=0.1, z_size=0.1, writer=None):
    try:
        
        pv.set_plot_theme("document")
        pv.global_theme.window_size = [1024, 768]
        pv.global_theme.jupyter_backend = 'static'
        p = pv.Plotter()
        ifc_file = ifcopenshell.open(file_name)
        print(file_name)

        ### CONVERSION FROM IFC TO MESH ###
        print(f'Converting {file_name}...')

        start_time = time.time()
        all_meshes, element_information, x,y,z, dims = open3d_block_by_element(ifc_file)

        total_polygons = sum(len(mesh.triangles) for mesh in all_meshes)

        # combine all meshes into one mesh and find the bounds
        combined_mesh = reduce(lambda m1, m2: m1 + m2, all_meshes)

        end_time = time.time()
        conversion_time = end_time - start_time

        ### POINT CLOUD CREATION ###
        print(f'Point cloud formation representing {file_name}...')
        start_time = time.time()
        point_cloud_list  = create_point_cloud(all_meshes, x_size)
        
        #merge all the list of point clouds into one
        pcd_pv = pv.PolyData()
        pcd_pv.points = np.vstack(list(map(lambda x: x.points, point_cloud_list)))
        sampling_points = len(pcd_pv.points)
        end_time = time.time()
        point_cloud_processing = end_time - start_time

        print(f'Rasterizing {file_name}...')
        start_time = time.time()

        # Compute the oriented bounding box (OBB)
        bb = combined_mesh.get_axis_aligned_bounding_box()
        
        # Extract the eight corner points of the OBB
        bb_points = np.asarray(bb.get_box_points())

        # Compute the axis-aligned bounding box of these eight points
        xmin, ymin, zmin = np.min(bb_points, axis=0)
        xmax, ymax, zmax = np.max(bb_points, axis=0)

        bounds = np.array([xmin, xmax, ymin, ymax, zmin, zmax])
        
        ### RASTERIZATION ###
        
        grid = voxelize_space(bounds, point_cloud_list, x_size, y_size, z_size)
        end_time = time.time()
        rasterization_time = end_time - start_time
        
        end_time = time.time()
        rasterization_time = end_time - start_time        

        ### VISUALIZATION ###
        print(f'Visualizing {file_name}...')
        start_time = time.time()
         
        # Extract the cells with non-empty attribute values
        non_empty_cells = grid.cell_data['attributes'] != -1
        
        # Create a box from the bounds
        box = pv.Box(bounds=bounds)

        # Add the box to the plotter
        p.add_mesh(box, color="blue", style="wireframe", line_width=2)
        # Add the mesh to the plotter with the non-empty cells
        p.add_mesh(grid.extract_cells(non_empty_cells), opacity=0.8,show_edges=False,show_scalar_bar=False)
        
        screenshot_filename = os.path.join("screenshots", f"{file_name}.png")

        p.screenshot(screenshot_filename)
        end_time = time.time()
        visualization_time = end_time - start_time
        #p.show()
         # Store the results in a dictionary
        results = {}
        results['File'] = file_name
        results['# of Polygons'] = total_polygons
        results['# of Points'] = sampling_points
        results['Cell Size (m)'] = x_size, y_size, z_size
        results['Conversion (sec)'] = round(conversion_time,2)
        results['Point Cloud Creation (sec)'] = round(point_cloud_processing,2)
        results['Rasterization (sec)'] = round(rasterization_time,2)
        results['Visualization (sec)'] = round(visualization_time,2)
        
        element_counts = fetch_element_quantities(ifc_file, grid, element_information)

        '''# Write results to Excel sheets
        if writer:
            # Write to 'Results' sheet
            results_df = pd.DataFrame([results])
            results_df.to_excel(writer, sheet_name='Results', startrow=writer.sheets['Results'].max_row, index=False, header=False)
            
            # Write to 'QTO' sheet
            element_counts_df = pd.DataFrame([element_counts])
            element_counts_df.to_excel(writer, sheet_name='QTO', startrow=writer.sheets['QTO'].max_row, index=False, header=False)'''

        return results, element_counts, grid

    except FileNotFoundError:
        print(f"Error: {file_name} not found.")
        print("Here's the traceback:")
        traceback.print_exc()
        return {}, {}

    except MemoryError:
        print(f"Error: Insufficient memory to process {file_name}.")
        print("Here's the traceback:")
        traceback.print_exc()
        return {}, {}

    except (TypeError, ValueError) as e:
        print(f"Error processing {file_name}: {str(e)}")
        print("Here's the traceback:")
        traceback.print_exc()
        return {}, {}

    except Exception as e:
        print(f"An unexpected error occurred while processing {file_name}: {str(e)}")
        print("Here's the traceback:")
        traceback.print_exc()
        return {}, {}
    
def find_ifc_files(directory):
    """Find all IFC files in a given directory."""
    # Use os.path.join to ensure the path is constructed correctly for any OS
    search_path = os.path.join(directory, "*.ifc")
    return glob.glob(search_path)

In [37]:
def group_contiguous_cells(grid_attributes, dims):
    # Initialize variables
    group_id = -3
    group_map = {}  # Dictionary to store the group id for each cell
    visited = set()  # Set to keep track of visited cells

    # Define a helper function for DFS
    def dfs(x, y, z, current_group):
        if (x, y, z) in visited or not (0 <= x < dims[0]-1 and 0 <= y < dims[1]-1 and 0 <= z < dims[2]-1):
            return
        if grid_attributes[x, y, z] != -2:
            return

        visited.add((x, y, z))
        group_map[(x, y, z)] = current_group

        # Explore neighbors (6 directions in a 3D grid)
        dfs(x+1, y, z, current_group)
        dfs(x-1, y, z, current_group)
        dfs(x, y+1, z, current_group)
        dfs(x, y-1, z, current_group)
        dfs(x, y, z+1, current_group)
        dfs(x, y, z-1, current_group)

    # Iterate over each cell and apply DFS
    for x in range(dims[0]-1):
        for y in range(dims[1]-1):
            for z in range(dims[2]-1):
                if grid_attributes[x, y, z] == -2 and (x, y, z) not in visited:
                    dfs(x, y, z, group_id)
                    group_id += -1
    return group_map

def dimension_analysis(group_map, min_size):
    # Initialize a dictionary to store the bounds of each group
    group_bounds = {}

    # Calculate bounds for each group
    for (x, y, z), group_id in group_map.items():
        if group_id not in group_bounds:
            group_bounds[group_id] = {'min_x': x, 'max_x': x, 'min_y': y, 'max_y': y, 'min_z': z, 'max_z': z}
        else:
            bounds = group_bounds[group_id]
            bounds['min_x'] = min(bounds['min_x'], x)
            bounds['max_x'] = max(bounds['max_x'], x)
            bounds['min_y'] = min(bounds['min_y'], y)
            bounds['max_y'] = max(bounds['max_y'], y)
            bounds['min_z'] = min(bounds['min_z'], z)
            bounds['max_z'] = max(bounds['max_z'], z)

    # Filter groups based on minimum size
    valid_groups = set()
    for group_id, bounds in group_bounds.items():
        if (bounds['max_x'] - bounds['min_x'] >= min_size['x'] and
            bounds['max_y'] - bounds['min_y'] >= min_size['y'] and
            bounds['max_z'] - bounds['min_z'] >= min_size['z']):
            valid_groups.add(group_id)

    return valid_groups

def find_rooms(grid, file_name):
    p2 = pv.Plotter()
    dims = grid.dimensions
    print(dims)
    grid_attributes = grid.cell_data['attributes'].reshape(dims[0]-1, dims[1]-1, dims[2]-1, order='F')
    try:
        # Check if the cell is contained in the x direction
        for z in range(dims[2]-1):
            for y in range(dims[1]-1):
                check = False  # Reset the check flag for each new row
                x = 0
                while x < dims[0]-1:
                    if grid_attributes[x, y, z] != -1:
                        check = True  # Set the flag when a filled cell is found
                    elif check and grid_attributes[x, y, z] == -1:
                        # Check the rest of the row for filled cells
                        for nx in range(x, dims[0]-1):
                            if grid_attributes[nx, y, z] != -1:
                                grid_attributes[x, y, z] = -2
                                break
                    x += 1  # Move to the next cell
        
        # Check if the cell marked with -2 is contained in the z direction
        for x in range(dims[0]-1):
            for y in range(dims[1]-1):
                z_check = False
                vertical_check = False
                z = 0
                while z < dims[2]-1:
                    if grid_attributes[x, y, z] not in [-1, -2]:
                        z_check = True  # Set the flag when a filled cell is found
                    elif z_check and grid_attributes[x, y, z] == -2:
                        for nz in range(z, dims[2]-1):
                            if grid_attributes[x, y, nz] not in [-1, -2]:
                                vertical_check = True
                                break
                    elif not vertical_check and not z_check:
                        if grid_attributes[x, y, z] == -2:
                            grid_attributes[x, y, z] = -1    
                    z += 1

        # Check if the cell is contained in the y direction
        for z in range(dims[2]-1):
            for x in range(dims[0]-1):
                y_check = False
                horizontal_check = False
                y = 0
                while y < dims[1]-1:
                    if grid_attributes[x, y, z] not in [-1, -2]:
                        y_check = True  # Set the flag when a filled cell is found
                    elif y_check and grid_attributes[x, y, z] == -2:
                        for ny in range(y, dims[1]-1):
                            if grid_attributes[x, ny, z] not in [-1, -2]:
                                horizontal_check = True
                                break
                    elif not horizontal_check and not y_check:
                        if grid_attributes[x, y, z] == -2:
                            grid_attributes[x, y, z] = -1   
                    y += 1
        
        group_map = group_contiguous_cells(grid_attributes, dims)
        
        # Define minimum size for each dimension as the rooms have to be bigger than 1.5 m
        min_size = {'x': 1.5, 'y': 1.5, 'z': 1.5} 
        # Perform dimension analysis
        valid_groups = dimension_analysis(group_map, min_size)
        spaces = []

        # Update grid attributes to keep only valid groups
        for (x, y, z), group_id in group_map.items():
            if group_id not in valid_groups:
                grid_attributes[x, y, z] = -1  # Reset cells not in valid groups
            else:
                grid_attributes[x, y, z] = group_id
                if group_id not in spaces:
                    spaces.append(group_id)

        # Update the grid with the new attributes
        grid.cell_data['attributes'] = grid_attributes.flatten(order='F')

        for space_no in spaces:
            p2.add_mesh(grid.extract_cells(grid.cell_data['attributes'] == space_no), color=random_color(), opacity=0.2, line_width=2)   
        
        #mask = np.logical_and(grid.cell_data['attributes'] != -1, grid.cell_data['attributes'] != -2)
        #p2.add_mesh(grid.extract_cells(mask), color="red", opacity=0.2, line_width=2)
        screenshot_filename = os.path.join("screenshots", f"{file_name}_rooms.png")
        p2.screenshot(screenshot_filename)

    except:
        print("No enclosed spaces found: ...")

    return grid

In [38]:
cell_sizes = [round(i, 2) for i in list(np.arange(0.5, 0.51, 0.05))]
cell_sizes = sorted(cell_sizes, reverse=True)  
parameter_sets = [
    {'x_size': x, 'y_size': x, 'z_size': z}
    for x in cell_sizes
    for z in cell_sizes
]

file_name = "IFC Files/02_Duplex.ifc"

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as a string: YYYYMMDD_HHMMSS
formatted_time = now.strftime("%Y%m%d")

# Create a filename using the formatted time
filename = f"output_{formatted_time}_rooms.txt"

# Open the file once and write results for each parameter set
with open(filename, 'a') as f:
    # Process the IFC file with each set of parameters and write the results to the text file
    for params in parameter_sets:
        results1, results2, grid = process_ifc_file(file_name, params['x_size'], params['y_size'], params['z_size'])

        # Convert the dictionaries to lists of strings
        results1_list = [str(res1) + " : " + str(results1.get(res1)) for res1 in results1]
        results2_list = [str(res2) + " : " + str(results1.get(res2)) for res2 in results2]
        # Append results to the text file
        f.write(','.join(results1_list) + '\n')
        f.write(','.join(results2_list) + '\n')

        print(results1, results2)

        #Find the enclosed spaces
        grid_with_rooms = find_rooms(grid, file_name)

IFC Files/02_Duplex.ifc
Converting IFC Files/02_Duplex.ifc...
Point cloud formation representing IFC Files/02_Duplex.ifc...
Rasterizing IFC Files/02_Duplex.ifc...
Visualizing IFC Files/02_Duplex.ifc...
Calculating quantities in <ifcopenshell.file.file object at 0x00000233E3F77100>...
{'File': 'IFC Files/02_Duplex.ifc', '# of Polygons': 9420, '# of Points': 863543, 'Cell Size (m)': (0.5, 0.5, 0.5), 'Conversion (sec)': 2.83, 'Point Cloud Creation (sec)': 0.15, 'Rasterization (sec)': 3.58, 'Visualization (sec)': 0.2} {'Quantity Calculation (sec)': 0.02, 'ZB_AUSWAHL_to_Element_Type': {}}
(55, 20, 18)
